# Personalize Workshop Cleanup - Lab 10 (optional)

This notebook will walk through deleting all of the resources created by the Personalization Labs in this workshop. You should only need to perform these steps if you have deployed the Retail Demo Store in your own AWS account and want to deprovision the Personalize resources. If you are participating in an AWS-led workshop, this process is likely not necessary.

Resources have to deleted in a specific sequence to avoid dependency errors. In order, we will delete recommenders and campaigns, solutions, event trackers, filters, datasets, and the dataset group. In addition, we need to make sure that each resource type is fully deleted before moving on to the next resource type. We'll also delete the schemas for our datasets and reset the SSM parameter values that are used by the Recommendations service and Web UI for personalization features.

We will be leveraging a utility module written on python that provides an orderly delete process for deleting all resources in each dataset group.

## Import depedencies and adjust path

The following code cell ensures that we are working with the latest version of the boto3 library and that the path is updated so we can load the `delete_dataset_groups` module.

In [ ]:
import logging
import os
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade --no-deps --force-reinstall botocore
sys.path.insert(0, os.path.abspath('../../src/aws-lambda/personalize-pre-create-resources'))

## Reset SSM parameters

Before deleting resources, we will reset the SSM parameter values so that the Recommendations service will no longer attempt to make requests to deleted resources.

In [ ]:
import boto3

param_names = [
    '/retaildemostore/personalize/event-tracker-id'
    '/retaildemostore/personalize/filters/filter-purchased-arn',
    '/retaildemostore/personalize/filters/filter-cstore-arn',
    '/retaildemostore/personalize/filters/filter-purchased-and-cstore-arn',
    '/retaildemostore/personalize/filters/promoted-items-filter-arn',
    '/retaildemostore/personalize/filters/promoted-items-no-cstore-filter-arn',
    '/retaildemostore/personalize/recommended-for-you-arn',
    '/retaildemostore/personalize/popular-items-arn',
    '/retaildemostore/personalize/related-items-arn',
    '/retaildemostore/personalize/personalized-ranking-arn',
    '/retaildemostore/personalize/personalized-offers-arn'
]

ssm = boto3.client('ssm')

for param_name in param_names:
    ssm.put_parameter(
        Name=param_name,
        Value='NONE',
        Type='String',
        Overwrite=True
    )
    
print('SSM parameters have been reset')

## Prepare and delete dataset groups

We are now ready to delete the active workshop dataset groups created by the labs involving Amazon Personalize. The `delete_dataset_groups` function will delete all dependent resources within the dataset groups starting with recommenders and campaigns, then solutions and datasets, filters and event trackers, and ending with schemas and the dataset groups themselves.

### Identify active workshop dataset groups

Let's start by checking the active dataset groups in the current account and check which of the two potential dataset groups are active. This will tell us which dataset group(s) need to be deleted.

In [ ]:
personalize = boto3.client('personalize')

# Dataset group names that may have been created by workshop labs.
possible_dataset_groups = [
    'retaildemostore-products',
    'retaildemostore-offers'
]

# Actual workshop dataset groups that exist. The logic below will add to this list.
active_dataset_groups = []

# Check dataset groups to see which workshop dataset groups were actually created.
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        if dataset_group['name'] in possible_dataset_groups:
            active_dataset_groups.append(dataset_group['name'])

print(f'Active workshop dataset groups that need to be deleted: {active_dataset_groups}')

### Prepare logging output

Next we will import the [delete_dataset_groups](../../src/aws-lambda/personalize-pre-create-resources/delete_dataset_groups.py) module and update the logging in the module so we can see its output here in the notebook.

In [ ]:
import delete_dataset_groups

In [ ]:
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)

delete_dataset_groups.logger.setLevel(logging.INFO)
delete_dataset_groups.logger.addHandler(handler)

### Delete dataset groups

Now we can delete the active workshop dataset groups. This can take several minutes depending on the number of dataset groups and resources within each dataset group. The function below will log its progress until finished.

In [ ]:
%%time

delete_dataset_groups.delete_dataset_groups(
    dataset_group_names = active_dataset_groups, 
    wait_for_resources = True
)

## Cleanup Complete

All resources created by the Personalize workshop(s) have been deleted.